In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import simpy
import simpy.rt  # Importando o RealtimeEnvironment
import random
from datetime import datetime, timedelta

In [2]:
%reload_ext watermark
%watermark -a "Rodrigo W Pisaia" --iversions --python

Author: Rodrigo W Pisaia

Python implementation: CPython
Python version       : 3.10.14
IPython version      : 8.27.0

numpy : 2.1.1
pandas: 2.2.3
tqdm  : 4.66.5
simpy : 4.1.1



In [3]:
# factory parameters
wood_capacity = 500
initial_wood = 100
electronic_capacity = 200
initial_electronic = 200
pre_paint_capacity = 100
post_paint_capacity = 200
dispatch_capacity = 500

In [4]:
#statistical
num_body = 2
mean_body = 1
std_body = 0.1

num_neck = 1
mean_neck = 1
std_neck = 0.2

num_paint = 1
mean_paint = 4
std_paint = 0.3

num_ensam = 4
mean_ensam = 1
std_ensam = 0.2

wood_critical_stock = (((8 / mean_body) * num_body + (8 / mean_neck) * num_neck) * 3)
electronic_critical_stock = (8 / mean_ensam) * num_ensam * 2

print(wood_critical_stock)
print(electronic_critical_stock)

72.0
64.0


In [5]:
class Guitar_Factory:
    def __init__(
        self,
        env,
        wood_capacity,
        initial_wood,
        electronic_capacity,
        initial_electronic,
        pre_paint_capacity,
        post_paint_capacity,
        dispatch_capacity,
        wood_critical_stock,
        electronic_critical_stock,
        dispatch_interval,
    ):
        self.env = env
        self.wood = simpy.Container(env, capacity=wood_capacity, init=initial_wood)
        self.wood_control = env.process(self.wood_stock_control(env))
        self.electronic = simpy.Container(
            env, capacity=electronic_capacity, init=initial_electronic
        )
        self.electronic_control = env.process(self.electronic_stock_control(env))
        self.pre_paint = simpy.Container(env, capacity=pre_paint_capacity, init=0)
        self.post_paint = simpy.Container(env, capacity=post_paint_capacity, init=0)
        self.dispatch = simpy.Container(env, capacity=dispatch_capacity, init=0)
        self.dispatch_interval = dispatch_interval

        env.process(self.dispatch_control(env))

    def wood_stock_control(self, env):
        yield env.timeout(0)
        while True:
            if self.wood.level <= wood_critical_stock:
                print(
                    "wood stock below critical level ({0}) at day {1}, hour {2}".format(
                        self.wood.level, int(env.now / 8), env.now % 8
                    )
                )
                print("calling wood supplier")
                yield env.timeout(16)
                print(
                    "wood supplier arrives at day {0}, hour {1}".format(
                        int(env.now / 8), env.now % 8
                    )
                )
                yield self.wood.put(300)
                print("new wood stock is {0}".format(self.wood.level))
                yield env.timeout(8)
            else:
                yield env.timeout(1)

    def electronic_stock_control(self, env):
        yield env.timeout(0)
        while True:
            if self.electronic.level <= electronic_critical_stock:
                print(
                    "electronic stock below critical level ({0}) at day {1}, hour {2}".format(
                        self.electronic.level, int(env.now / 8), env.now % 8
                    )
                )
                print("calling electronic supplier")
                yield env.timeout(9)
                print(
                    "electronic supplier arrives at day {0}, hour {1}".format(
                        int(env.now / 8), env.now % 8
                    )
                )
                yield self.electronic.put(30)
                print("new electronic stock is {0}".format(self.electronic.level))
                yield env.timeout(8)
            else:
                yield env.timeout(1)

    def dispatch_control(self, env):
        while True:
            # aguarda a  hora de realizar o dispatch
            yield env.timeout(self.dispatch_interval)

            if self.dispatch.level < self.dispatch.capacity:
                yield self.dispatch.put(1)  # Adiciona um item ao dispatch
                print(
                    f"Dispatch realizado em {env.now}: {self.dispatch.level} items no dispatch"
                )
            else:
                print(f"Dispatch cheio em {env.now}: {self.dispatch.level} items")

In [6]:
def body_maker(env, guitar_factory, logs):
    while True:
        yield guitar_factory.wood.get(1)
        body_time = random.gauss(mean_body, std_body)
        yield env.timeout(body_time)
        guitar_factory.pre_paint.put(1)

        logs.append({
            'time': env.now,
            'event': 'body_made',
            'wood_stock': guitar_factory.wood.level,
            'pre_paint_stock': guitar_factory.pre_paint.level
        })

def neck_maker(env, guitar_factory, logs):
    while True:
        yield guitar_factory.wood.get(1)
        neck_time = random.gauss(mean_neck, std_neck)
        yield env.timeout(neck_time)
        guitar_factory.pre_paint.put(2)

        logs.append({
            'time': env.now,
            'event': 'neck_made',
            'wood_stock': guitar_factory.wood.level,
            'pre_paint_stock': guitar_factory.pre_paint.level
        })

def painter(env, guitar_factory, logs):
    while True:
        yield guitar_factory.pre_paint.get(10)
        paint_time = random.gauss(mean_paint, std_paint)
        yield env.timeout(paint_time)
        guitar_factory.post_paint.put(10)

        logs.append({
            'time': env.now,
            'event': 'painting_done',
            'post_paint_stock': guitar_factory.post_paint.level
        })

def assembler(env, guitar_factory, logs):
    while True:
        yield guitar_factory.post_paint.get(1)
        yield guitar_factory.electronic.get(1)
        assembling_time = max(random.gauss(mean_ensam, std_ensam), 1)
        yield env.timeout(assembling_time)
        guitar_factory.dispatch.put(1)

        logs.append({
            'time': env.now,
            'event': 'guitar_assembled',
            'dispatch_stock': guitar_factory.dispatch.level
        })

In [7]:
def body_maker_gen(env, guitar_factory, logs):
    for i in range(num_body):
        env.process(body_maker(env, guitar_factory, logs))
        yield env.timeout(0)

def neck_maker_gen(env, guitar_factory, logs):
    for i in range(num_neck):
        env.process(neck_maker(env, guitar_factory, logs))
        yield env.timeout(0)

def painter_maker_gen(env, guitar_factory, logs):
    for i in range(num_paint):
        env.process(painter(env, guitar_factory, logs))
        yield env.timeout(0)

def assembler_maker_gen(env, guitar_factory, logs):
    for i in range(num_ensam):
        env.process(assembler(env, guitar_factory, logs))
        yield env.timeout(0)

In [8]:
total_time_hours = 40  # Horas
logs = []

factor = 1
env = simpy.rt.RealtimeEnvironment(factor=factor, strict=True)
guitar_factory = Guitar_Factory(env, wood_capacity, initial_wood, electronic_capacity,
                                 initial_electronic, pre_paint_capacity, post_paint_capacity,
                                 dispatch_capacity, wood_critical_stock, electronic_critical_stock, dispatch_interval=10)

# init
body_gen = env.process(body_maker_gen(env, guitar_factory, logs))
neck_gen = env.process(neck_maker_gen(env, guitar_factory, logs))
painter_gen = env.process(painter_maker_gen(env, guitar_factory, logs))
assembler_gen = env.process(assembler_maker_gen(env, guitar_factory, logs))

total_time_hours = 120  # hours
env.run(until=total_time_hours)

wood stock below critical level (71) at day 1, hour 1
calling wood supplier
Dispatch realizado em 10: 11 items no dispatch
Dispatch realizado em 20: 32 items no dispatch
wood supplier arrives at day 3, hour 1
new wood stock is 321
Dispatch realizado em 30: 60 items no dispatch
Dispatch realizado em 40: 84 items no dispatch
Dispatch realizado em 50: 115 items no dispatch
Dispatch realizado em 60: 136 items no dispatch
electronic stock below critical level (62) at day 7, hour 5
calling electronic supplier
electronic supplier arrives at day 8, hour 6
Dispatch realizado em 70: 165 items no dispatch
new electronic stock is 70
electronic stock below critical level (50) at day 9, hour 6
calling electronic supplier
Dispatch realizado em 80: 188 items no dispatch
electronic supplier arrives at day 10, hour 7
new electronic stock is 60
Dispatch realizado em 90: 216 items no dispatch
electronic stock below critical level (40) at day 11, hour 7
calling electronic supplier
Dispatch realizado em 100

In [9]:
start_date = datetime(2024, 1, 1)
for log in logs:
    log['time'] = start_date + timedelta(hours=log['time'])

In [10]:
logs

[{'time': datetime.datetime(2024, 1, 1, 0, 46, 58, 360958),
  'event': 'neck_made',
  'wood_stock': 97,
  'pre_paint_stock': 2},
 {'time': datetime.datetime(2024, 1, 1, 0, 55, 59, 756361),
  'event': 'body_made',
  'wood_stock': 96,
  'pre_paint_stock': 3},
 {'time': datetime.datetime(2024, 1, 1, 1, 2, 26, 544242),
  'event': 'body_made',
  'wood_stock': 95,
  'pre_paint_stock': 4},
 {'time': datetime.datetime(2024, 1, 1, 1, 44, 5, 892271),
  'event': 'body_made',
  'wood_stock': 94,
  'pre_paint_stock': 5},
 {'time': datetime.datetime(2024, 1, 1, 2, 3, 26, 980102),
  'event': 'body_made',
  'wood_stock': 93,
  'pre_paint_stock': 6},
 {'time': datetime.datetime(2024, 1, 1, 2, 13, 4, 579983),
  'event': 'neck_made',
  'wood_stock': 92,
  'pre_paint_stock': 8},
 {'time': datetime.datetime(2024, 1, 1, 2, 40, 28, 551339),
  'event': 'body_made',
  'wood_stock': 91,
  'pre_paint_stock': 9},
 {'time': datetime.datetime(2024, 1, 1, 2, 52, 16, 712916),
  'event': 'body_made',
  'wood_stock': 9

In [11]:
# Salvar logs em DataFrame
logs_df = pd.DataFrame(logs)
logs_df.to_csv('production_logs.csv', index=False)
logs_df.tail(50)

,time,event,wood_stock,pre_paint_stock,post_paint_stock,dispatch_stock
617,2024-01-05 15:07:02.243608,guitar_assembled,NaN,NaN,NaN,271.0
618,2024-01-05 15:45:53.027064,neck_made,65.0,100.0,NaN,NaN
619,2024-01-05 15:48:45.251927,body_made,64.0,100.0,NaN,NaN
620,2024-01-05 15:54:52.598302,body_made,63.0,100.0,NaN,NaN
621,2024-01-05 16:19:08.691412,painting_done,NaN,NaN,10.0,NaN
622,2024-01-05 16:48:15.966481,body_made,62.0,99.0,NaN,NaN
623,2024-01-05 16:54:51.196399,body_made,61.0,99.0,NaN,NaN
624,2024-01-05 17:04:51.503479,neck_made,60.0,99.0,NaN,NaN
625,2024-01-05 17:19:08.691412,guitar_assembled,NaN,NaN,NaN,272.0
626,2024-01-05 17:19:08.691412,guitar_assembled,NaN,NaN,NaN,273.0


In [12]:
((1.96/2)*(1/0.05))**2

384.1600000000001

In [13]:
384.1600000000001*(0.5**2)

96.04000000000002